# Model: Random Forest

Até agora neste projeto, aprendemos como recuperar e descompactar dados, e como gerenciar dados desbalanceados para construir um modelo de decisão-tree.

Nesta lição, vamos expandir nosso modelo de decisão tree em uma floresta inteira (um exemplo de algo chamado de **ensemble model**); aprender como usar uma **grid search** para ajustar hiperparâmetros; e criar uma função que carrega dados e um modelo pré-treinado, e usa esse modelo para gerar uma série de previsões.

In [ ]:
import gzip
import json
import pickle
import pandas as pd
import matplotlib.pyplot as plt

from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline

# Preparando Dados

Como sempre, começaremos importando o dataset.

## Importação

### Exercício:
Complete a função `wrangle` abaixo usando o código que você desenvolveu nas ultimas lições. Em seguida, use-a para importar `poland-bankruptcy-data-2009.json.gz` para o DataFrame `df`.

In [ ]:
def wrangle(filename):

    return df

In [ ]:
df = ...
print(df.shape)
df.head()

## Divisão

### Exercício:
Crie sua matriz de características `X` e vetor alvo `y`. Seu alvo é `"bankrupt"`.

In [ ]:
target = "bankrupt"
X = ...
y = ...

print("X shape:", X.shape)
print("y shape:", y.shape)

Como não estamos trabalhando com dados de séries temporais, vamos dividir nosso dataset aleatoriamente em conjuntos de treinamento e teste

### Exercício:
Divida seus dados (`X` e `y`) em conjuntos de treinamento e teste usando uma divisão aleatória de train-test. Seu conjunto de teste deve ser 20% do seu total de dados. E não se esqueça de definir um `random_state` para reprodutibilidade.

In [ ]:
X_train, X_test, y_train, y_test = ...

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

Você pode ter notado que não criamos um conjunto de validação, embora estejamos planejando ajustar os hiperparâmetros do nosso modelo nesta lição. Isso porque vamos usar validação cruzada, sobre a qual falaremos mais adiante.

## Resample

### Exercício:
Crie uma nova matriz de características `X_train_over` e vetor alvo `y_train_over` realizando over-sampling aleatório nos dados de treinamento.

In [ ]:
over_sampler = ...
X_train_over, y_train_over = ...
print("X_train_over shape:", X_train_over.shape)
X_train_over.head()

# Construindo Model

Agora que temos nossos dados configurados da maneira correta, podemos construir o modelo. 🏗

## Baseline

### Exercício:
Calcule a pontuação de precisão básica para o seu modelo.

In [ ]:
acc_baseline = ...
print("Baseline Accuracy:", round(acc_baseline, 4))

## Iteratar

Até agora, construímos modelos únicos que preveem um único resultado. Essa definitivamente é uma maneira útil de prever o futuro, mas e se o modelo que construímos não for o *certo*? Se pudéssemos de alguma forma usar mais de um modelo simultaneamente, teríamos uma previsão mais confiável.

**Modelos de conjunto** funcionam construindo múltiplos modelos em subconjuntos aleatórios dos mesmos dados e, em seguida, comparando suas previsões para fazer uma previsão final. Como usamos uma árvore de decisão na última lição, vamos criar um conjunto de árvores aqui. Esse tipo de modelo é chamado de **random forest**.

Começaremos criando um pipeline para simplificar nosso fluxo de trabalho.

Crie um pipeline chamado `clf` (abreviação de "classifier") que contenha um transformador `SimpleImputer` e um preditor `RandomForestClassifier`.

In [ ]:
clf = ...
print(clf)

Por padrão, o número de árvores em nossa floresta (`n_estimators`) é definido como 100. Isso significa que, quando treinamos esse classificador, estaremos ajustando 100 árvores. Embora leve mais tempo para treinar, isso deve levar a um desempenho melhor.

Para obter o melhor desempenho do nosso modelo, precisamos ajustar seus hiperparâmetros. Mas como podemos fazer isso se não criamos um conjunto de validação? A resposta é **validação cruzada**. Portanto, antes de analisarmos os hiperparâmetros, vamos ver como a validação cruzada funciona com o classificador que acabamos de construir.

###Exercício:
Realize a validação cruzada com seu classificador, usando os dados de treinamento sobre-amostrados. Queremos cinco dobras, então defina `cv` como 5. Também queremos acelerar o treinamento, então defina `n_jobs` como -1.

In [ ]:
cv_acc_scores = ...
print(cv_acc_scores)

Isso levou um bom tempo, mas acabamos de treinar 500 classificadores de random forest (100 trabalhos x 5 dobras). Não é de se admirar que tenha demorado tanto!

__Dica profissional:__ embora `cross_val_score` seja útil para ter uma ideia de como a validação cruzada funciona, você raramente a usará. Em vez disso, a maioria das pessoas inclui um argumento `cv` ao fazer uma busca de hiperparâmetros.

### Exercício:
Agora que temos uma ideia de como a validação cruzada funciona, vamos ajustar nosso modelo. O primeiro passo é criar uma faixa de hiperparâmetros que queremos avaliar.

Crie um dicionário com a faixa de hiperparâmetros que queremos avaliar para nosso classificador.

1. Para o `SimpleImputer`, experimente as estratégias `"mean"` e `"median"`.
2. Para o `RandomForestClassifier`, experimente configurações de `max_depth` entre 10 e 50, em incrementos de 10.
3. Também para o `RandomForestClassifier`, experimente configurações de `n_estimators` entre 25 e 100, em incrementos de 25.

In [ ]:
params = ...
params

Agora que temos nossa grade de hiperparâmetros, vamos incorporá-la em uma **grid search**.

### Exercício:
Crie um `GridSearchCV` chamado `model` que inclua seu classificador e grade de hiperparâmetros. Certifique-se de usar os mesmos argumentos para `cv` e `n_jobs` que você usou acima, e defina `verbose` como 1.

In [ ]:
model = ...
model

Finalmente, agora vamos ajustar o modelo.

### Exercício:
Ajuste `model` aos dados de treinamento **over-sampled**.

In [ ]:
# Train model


Isso levará algum tempo para treinar, então vamos aproveitar um momento para pensar sobre o porquê. Quantas florestas acabamos de testar? 4 diferentes `max_depth`s vezes 3 `n_estimators` vezes 2 estratégias de imputação... isso totaliza 24 florestas. Quantas adaptações acabamos de fazer? 24 florestas vezes 5 dobras é 120. E lembre-se de que cada floresta é composta por 25-75 árvores, o que resulta em *pelo menos* 3.000 árvores. Portanto, é computacionalmente caro!

Ok, agora que testamos todos esses modelos, vamos dar uma olhada nos resultados.

### Exercício:
Extraia os resultados da validação cruzada de `model` e carregue-os em um DataFrame chamado `cv_results`.

In [ ]:
cv_results = ...
cv_results.head(10)

Além das pontuações de precisão para todos os diferentes modelos que tentamos durante nossa busca em grade, podemos ver quanto tempo levou para cada modelo ser treinado. Vamos dar uma olhada mais de perto em como as diferentes configurações de hiperparâmetros afetam o tempo de treinamento.

Primeiro, vamos olhar para `n_estimators`. Nossa busca em grade avaliou esse hiperparâmetro para várias configurações de `max_depth`, mas vamos focar apenas em modelos onde `max_depth` é igual a 10.

### Exercício:
Crie uma máscara para `cv_results` para as linhas onde `"param_randomforestclassifier__max_depth"` é igual a 10. Em seguida, plote `"param_randomforestclassifier__n_estimators"` no eixo x e `"mean_fit_time"` no eixo y. Não se esqueça de rotular seus eixos e incluir um título.

In [ ]:
# Create mask
mask = ...
# Plot fit time vs n_estimators

# Label axes
plt.xlabel("Number of Estimators")
plt.ylabel("Mean Fit Time [seconds]")
plt.title("Training Time vs Estimators (max_depth=10)");

Em seguida, vamos analisar `max_depth`. Aqui, também vamos limitar nossos dados às linhas onde `n_estimators` é igual a 25.

### Exercício:
Crie uma máscara para `cv_results` para as linhas onde `"param_randomforestclassifier__n_estimators"` é igual a 25. Em seguida, plote `"param_randomforestclassifier__max_depth"` no eixo x e `"mean_fit_time"` no eixo y. Não se esqueça de rotular seus eixos e incluir um título.

In [ ]:
# Create mask
mask = ...
# Plot fit time vs max_depth

# Label axes
plt.xlabel("Max Depth")
plt.ylabel("Mean Fit Time [seconds]")
plt.title("Training Time vs Max Depth (n_estimators=25)");

Há uma tendência geral de aumento, mas vemos muitas oscilações aqui. Isso acontece porque, para cada profundidade máxima, a busca em grade tenta duas estratégias de imputação diferentes: média e mediana. A mediana é muito mais rápida de calcular, o que acelera o tempo de treinamento.

Finalmente, vamos olhar para os hiperparâmetros que levaram ao melhor desempenho.

Extraia os melhores hiperparâmetros de `model`.

In [ ]:
# Extract best hyperparameters


Observe que não precisamos construir e treinar um novo modelo com essas configurações. Agora que a busca em grade está completa, quando usamos `model.predict()`, ele fornecerá previsões usando o melhor modelo — algo que faremos ao final desta lição.

## Avaliar

Certo: o momento da verdade. Vamos ver como nosso modelo se desempenha.

### Exercício:
Calcule as pontuações de precisão para treinamento e teste de `model`.

In [ ]:
acc_train = ...
acc_test = ...

print("Training Accuracy:", round(acc_train, 4))
print("Test Accuracy:", round(acc_test, 4))

Superamos a linha de base! Apenas por pouco, mas superamos.

Em seguida, vamos usar uma matriz de confusão para ver como nosso modelo se desempenha. Para entender melhor os valores que veremos na matriz, vamos primeiro contar quantas observações em nosso conjunto de teste pertencem às classes positiva e negativa.

In [ ]:
y_test.value_counts()

### Exercício:
Plote uma matriz de confusão que mostre como seu melhor modelo se desempenha em seu conjunto de teste.

In [ ]:
# Plot confusion matrix


Observe a relação entre os números nesta matriz e a contagem que você fez na tarefa anterior. Se você somar os valores na linha inferior, obterá o total de observações positivas em `y_test` ($72 + 11 = 83$). E a soma da linha superior corresponde ao número de observações negativas ($1903 + 10 = 1913$).

# Communicar Resultados

### Exercício:
Crie um gráfico de barras horizontal com os 10 recursos mais importantes para o seu modelo.

In [ ]:
# Get feature names from training data
features = ...
# Extract importances from model
importances = ...
# Create a series with feature names and importances
feat_imp = ...
# Plot 10 most important features

plt.xlabel("Gini Importance")
plt.ylabel("Feature")
plt.title("Feature Importance");

A única coisa que resta agora é salvar seu modelo para que ele possa ser reutilizado.

Usando um gerenciador de contexto, salve seu modelo com melhor desempenho em um arquivo chamado `"model-5-3.pkl"`.

In [ ]:
# Save model


### Exercício:
Crie uma função `make_predictions`. Ela deve aceitar dois argumentos: o caminho de um arquivo JSON que contém dados de teste e o caminho de um modelo serializado. A função deve carregar e limpar os dados usando a função `wrangle` que você criou, carregar o modelo, gerar um array de previsões e converter esse array em uma Série. (A Série deve ter o nome `"bankrupt"` e os mesmos rótulos de índice que os dados de teste.) Finalmente, a função deve retornar suas previsões como uma Série.

In [ ]:
def make_predictions(data_filepath, model_filepath):
    # Wrangle JSON file
    X_test = ...
    # Load model

    # Generate predictions
    y_test_pred = ...
    # Put predictions into Series with name "bankrupt", and same index as X_test
    y_test_pred = ...
    return y_test_pred

### Exercício:
Use o código abaixo para verificar sua função `make_predictions`. Assim que estiver satisfeito com o resultado, envie-o para o avaliador.

In [ ]:
y_test_pred = make_predictions(
    data_filepath="data/poland-bankruptcy-data-2009-mvp-features.json.gz",
    model_filepath="model-5-3.pkl",
)

print("predictions shape:", y_test_pred.shape)
y_test_pred.head()

# Gradient Boosting Trees

Você tem trabalhado duro e agora tem todas as ferramentas necessárias para construir e ajustar modelos. Vamos começar esta lição da mesma forma que começamos as outras: preparando os dados e construindo nosso modelo, e desta vez com um novo modelo de conjunto. Assim que estiver funcionando, aprenderemos algumas novas métricas de desempenho para avaliá-lo. Ao final desta lição, você terá escrito seu primeiro módulo em Python!

In [ ]:
import gzip
import json
import pickle

import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact
from teaching_tools.widgets import ConfusionMatrixWidget

from imblearn.over_sampling import RandomOverSampler

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    classification_report,
    confusion_matrix,
)
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline

# Preparando Dados

Toda a preparação de dados para este módulo é a mesma que foi da última vez. Vejo você do outro lado!

## Importação

### Exercício:
Complete a função `wrangle` abaixo usando o código que você desenvolveu nas ultimas lições. Em seguida, use-a para importar `poland-bankruptcy-data-2009.json.gz` no DataFrame `df`.

In [ ]:
def wrangle(filename):

    return df

In [ ]:
df = ...
print(df.shape)
df.head()

## Divisão

### Exercício:
Crie sua matriz de features `X` e o vetor alvo `y`. Seu alvo é `"bankrupt"`.

In [ ]:
target = "bankrupt"
X = ...
y = ...

print("X shape:", X.shape)
print("y shape:", y.shape)

### Exercício:
Divida seus dados (`X` e `y`) em conjuntos de treinamento e teste usando uma divisão aleatória com `train-test split`. Seu conjunto de teste deve ser 20% do total dos dados. E não se esqueça de definir um `random_state` para reprodutibilidade.

In [ ]:
X_train, X_test, y_train, y_test = ...

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

## Resample

### Exercicio:
Crie uma nova matriz de features `X_train_over` e um vetor alvo `y_train_over` realizando random over-sampling nos dados de treinamento.

In [ ]:
over_sampler = ...
X_train_over, y_train_over = ...
print("X_train_over shape:", X_train_over.shape)
X_train_over.head()

# Construindo Model

Agora vamos montar nosso modelo. Começaremos calculando a baseline accuracy, assim como fizemos da última vez.

## Baseline

### Exercício:
Calcule a baseline accuracy para o seu modelo.

In [ ]:
acc_baseline = ...
print("Baseline Accuracy:", round(acc_baseline, 4))

## Iterar

Embora os blocos de construção sejam os mesmos, é aqui que começamos a trabalhar com algo novo. Primeiro, vamos usar um novo tipo de modelo de ensemble para o nosso classificador.

Crie um pipeline chamado `clf` (abreviação de "classifier") que contenha um transformador `SimpleImputer` e um preditor `GradientBoostingClassifier`.

In [ ]:
clf = ...

Lembre-se, enquanto fazemos isso, que queremos olhar apenas para a *classe positiva*. Aqui, a classe positiva é aquela em que as empresas realmente faliram. No dicionário que fizemos na última vez, a classe positiva é composta pelas empresas com o par chave-valor `bankrupt: true`.

Em seguida, vamos ajustar alguns dos hiperparâmetros do nosso modelo.

### Exercício:
Crie um dicionário com a faixa de hiperparâmetros que queremos avaliar para o nosso classificador.

1. Para o `SimpleImputer`, experimente as estratégias `"mean"` e `"median"`.
2. Para o `GradientBoostingClassifier`, tente configurações de `max_depth` entre 2 e 5.
3. Também para o `GradientBoostingClassifier`, tente configurações de `n_estimators` entre 20 e 31, em passos de 5.

In [ ]:
params = ...
params

### Exercício:
Observe que estamos tentando números muito menores de `n_estimators`. Isso ocorre porque o `GradientBoostingClassifier` é mais lento para treinar do que o `RandomForestClassifier`. Você pode tentar aumentar o número de estimadores para ver se o desempenho do modelo melhora, mas lembre-se de que você pode esperar um longo tempo!

### Exercício:
Crie um `GridSearchCV` chamado `model` que inclua seu classificador e a grade de hiperparâmetros. Certifique-se de usar os mesmos argumentos para `cv` e `n_jobs` que você usou anteriormente, e defina `verbose` como 1.

In [ ]:
model = ...

Agora que temos tudo o que precisamos para o modelo, vamos ajustá-lo aos dados e ver o que conseguimos.

### Exercício:
Ajuste seu `model` aos dados de treinamento que foram superamostrados.

In [ ]:
# Fit model to over-sampled training data


### Exercício:
Extraia os resultados da validação cruzada do `model` e carregue-os em um DataFrame chamado `cv_results`.

In [ ]:
results = ...
results.sort_values("rank_test_score").head(10)

Há vários hiperparâmetros lá, então vamos extrair aqueles que funcionam melhor para o nosso modelo.

### Exercício:
Extraia os melhores hiperparâmetros do `model`.

In [ ]:
# Extract best hyperparameters


# Avaliar

Agora que temos um modelo funcional que está realmente nos fornecendo algo útil, vamos ver quão bom ele realmente é.

### Exercício:
Calcule as pontuações de acurácia de treinamento e teste para o `model`.

In [ ]:
acc_train = ...
acc_test = ...

print("Training Accuracy:", round(acc_train, 4))
print("Validation Accuracy:", round(acc_test, 4))

Assim como antes, vamos criar um `ConfusionMatrix` para ver como nosso modelo está fazendo suas previsões corretas e incorretas.

### Exercício:
Plote uma matriz de confusão que mostre como o seu melhor modelo está se saindo no seu conjunto de teste.

In [ ]:
# Plot confusion matrix


Esta matriz é um ótimo lembrete de como nossos dados estão desequilibrados e de por que a acurácia nem sempre é a melhor métrica para julgar se um modelo está nos fornecendo o que queremos. Afinal, se 95% das empresas em nosso conjunto de dados não faliram, tudo o que o modelo precisa fazer é sempre prever `{"falência": False}`, e estará certo 95% das vezes. A pontuação de acurácia será incrível, mas não nos dirá o que realmente precisamos saber.

Em vez disso, podemos avaliar nosso modelo usando duas novas métricas: **precisão** e **recall**. A pontuação de precisão é importante quando queremos que nosso modelo só preveja que uma empresa vai falir se estiver muito confiante em sua previsão. A pontuação de *recall* é importante se quisermos garantir que todas as empresas que falirão sejam identificadas, mesmo que isso signifique cometer alguns erros às vezes.

Vamos começar com um relatório que você pode criar com o scikit-learn para calcular ambas as métricas. Em seguida, analisaremos cada uma delas usando uma ferramenta de visualização que desenvolvemos especialmente para o Laboratório de Ciência de Dados.

### Exercício:
Imprima o relatório de classificação para o seu modelo, utilizando o conjunto de teste.

In [ ]:
# Print classification report
print(...)

### Exercício:
Execute a célula abaixo para carregar o widget da matriz de confusão.

In [ ]:
c = ConfusionMatrixWidget(model, X_test, y_test)
c.show()

Se você mover o limite de probabilidade, verá que há uma troca entre precisão e recall. Ou seja, à medida que um melhora, o outro sofre. Como cientista de dados, você muitas vezes precisará decidir se prefere um modelo com melhor precisão ou melhor recall. A escolha dependerá de como você pretende usar o modelo.

Vamos examinar dois exemplos, um em que o recall é a prioridade e outro em que a precisão é mais importante. Primeiro, imagine que você trabalha para uma agência reguladora da União Europeia que auxilia empresas e investidores a navegar por processos de insolvência. Você quer construir um modelo para prever quais empresas podem falir, para que possa enviar aos devedores informações sobre como solicitar proteção legal antes que a empresa se torne insolvente. Os custos administrativos de enviar informações para uma empresa são de €500. Os custos legais para o sistema judiciário europeu se uma empresa não solicitar proteção antes da falência são de €50.000.

Para um modelo como este, queremos focar no **recall**, pois o recall se trata de *quantidade*. Um modelo que prioriza recall lançará a rede mais ampla possível, o que é a abordagem ideal para esse problema. Queremos enviar informações para o maior número possível de empresas que possam falir, pois custa muito menos enviar informações para uma empresa que talvez não se torne insolvente do que ignorar uma que realmente falirá.

### Exercício:
Execute a célula abaixo e use o controle deslizante para alterar o limite de probabilidade do seu modelo. Qual relação você observa entre as mudanças no limite e as mudanças nos custos administrativos e legais desperdiçados? Na sua opinião, o que é mais importante para este modelo: alta precisão ou alto recall?

In [ ]:
c.show_eu()

Para o segundo exemplo, vamos supor que trabalhamos em uma firma de private equity que compra empresas em dificuldades, as melhora e depois as vende para obter lucro. Você quer construir um modelo para prever quais empresas vão falir para que possa comprá-las antes dos seus concorrentes. Se a firma compra uma empresa que realmente está insolvente, pode lucrar €100 milhões ou mais. Mas se comprar uma empresa que não está insolvente e não pode ser revendida com lucro, a firma perderá €250 milhões.

Para um modelo como este, queremos focar na **precisão**. Se estamos tentando maximizar nosso lucro, a *qualidade* de nossas previsões é muito mais importante do que a *quantidade* de nossas previsões. Não é um grande problema se não conseguirmos identificar todas as empresas insolventes, mas é *definitivamente* um grande problema se as empresas que identificamos não acabarem se tornando insolventes.

Desta vez, vamos construir a visualização juntos.

### Exercício:
Crie um painel interativo que mostre como o lucro e as perdas das empresas mudam em relação ao limite de probabilidade do seu modelo. Comece com a função `make_cnf_matrix`, que deve calcular e imprimir lucros/perdas e exibir uma matriz de confusão. Em seguida, crie um `FloatSlider` chamado `thresh_widget` que varia de 0 a 1. Por fim, combine sua função e o controle deslizante na função `interact`.

In [ ]:
def make_cnf_matrix(threshold):

    pass


thresh_widget = ...

interact(make_cnf_matrix, threshold=thresh_widget);

__Go Further:💡__ Alguns alunos sugeriram que este widget seria melhor se mostrasse a soma dos lucros e perdas. Você pode adicionar esse total?

# Comunicar Resultados

Quase lá! Salve o melhor modelo para que possamos compartilhá-lo com outras pessoas e, em seguida, junte tudo o que aprendemos na última lição.

### Exercício:
Usando um gerenciador de contexto, salve seu modelo de melhor desempenho em um arquivo chamado `"model-5-4.pkl"`.

In [ ]:
# Save model


### Exercício:
Abra o arquivo `my_predictor_lesson.py`, adicione as funções `wrangle` e `make_predictions` da última lição e insira todas as instruções de importação necessárias no topo do arquivo. Quando terminar, salve o arquivo. Você pode verificar se o conteúdo está correto executando a célula abaixo.

In [ ]:
%%bash

cat my_predictor_lesson.py

Parabéns! Você criou seu primeiro módulo!

### Exercício:
Importe sua função `make_predictions` do seu módulo `my_predictor` e use o código abaixo para garantir que ela funcione conforme o esperado. Quando estiver satisfeito, envie para o avaliador.

In [ ]:
# Import your module


# Generate predictions
y_test_pred = make_predictions(
    data_filepath="data/poland-bankruptcy-data-2009-mvp-features.json.gz",
    model_filepath="model-5-4.pkl",
)

print("predictions shape:", y_test_pred.shape)
y_test_pred.head()